# Import libraries and init

In [1]:
# import ROOT -- don't need this for now
import numpy as np
import pandas as pd
from collections import OrderedDict
import math

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams.update({'font.size': 9})

import uproot # uproot needs to be last for some reason

In [2]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

In [3]:
from functools import reduce
import operator

# Load ntuples from analyzer

Define parameters in separate block so can change them without loading files all over again

In [18]:
#masses = [('60p0','20p0'),('6p0','2p0'),('52p5','5p0'),('5p25','0p5')]
masses = [('60p0','20p0')]
printMasses = lambda mass: f'({float(mass[0].replace("p","."))-float(mass[1].replace("p","."))/2}, ' + \
                                f'{float(mass[0].replace("p","."))+float(mass[1].replace("p","."))/2}) GeV'
mchis = dict([(mass[0], printMasses(mass)) for mass in masses])
ctaus = [1,10, 100]#, 1000]

numCuts = np.arange(0,6)
labels = [ f'cut{cut}' for cut in numCuts ]
cutDescriptions = ['cut1: MET/MHT trigger fired (120 GeV)', \
                   'cut2: j1 pT > 120 GeV, <= 2j w/ pT > 30 GeV', \
                   'cut3: mu1 pT > 5 GeV, 0.1 < |dxy| < 700 cm', \
                   'cut4: mu2 pT > 5 GeV, 0.1 < |dxy| < 700 cm', \
                   r'cut5: $|\Delta\Phi$(MET, mu pair)| < 0.4'
                  ]

branchPath = 'SREffi_dsa'
baseDir = '/uscms/home/mreid/nobackup/signal_region/2018/pileup/'
#baseDir = '../iDM_analysis_skimming/washAOD/'
#baseDir = '/uscms/home/mreid/../../a/as2872/nobackup/iDM/AndreAnalysis/CMSSW_10_4_0_pre2/src/Firefighter/washAOD/SROptimization/'
#baseDir = '/uscms/home/mreid/nobackup/signal_region/2018/GenFilter_1or2Jets_longname/'
fileName = lambda Mchi, dMchi, ctau: baseDir + f'pileup_trackquality_Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

Load signal files

In [19]:
numParams = len(masses)*len(ctaus)
countParam = 1

trees = OrderedDict({})
treesglobal = OrderedDict({})
genInfo = OrderedDict({})
numEvents = OrderedDict({})
for (Mchi, dMchi) in masses:
    countCtaus = 1
    trees[Mchi] = OrderedDict({})
    treesglobal[Mchi] = OrderedDict({})
    numEvents[Mchi] = OrderedDict({})
    genInfo[Mchi] = OrderedDict({})
    for ctau in ctaus:
        genInfo[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))['GEN/gen']#.pandas.df(flatten=False)
        trees[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))[branchPath + f'/cutsTree']#.pandas.df(flatten=False)
        treesglobal[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))['SREffi_gbm' + f'/cutsTree']#.pandas.df(flatten=False)
        print(f'{countParam} of {numParams}: ' + fileName(Mchi, dMchi, ctau))
        countParam += 1

1 of 3: /uscms/home/mreid/nobackup/signal_region/2018/pileup/pileup_trackquality_Mchi-60p0_dMchi-20p0_ctau-1.root
2 of 3: /uscms/home/mreid/nobackup/signal_region/2018/pileup/pileup_trackquality_Mchi-60p0_dMchi-20p0_ctau-10.root
3 of 3: /uscms/home/mreid/nobackup/signal_region/2018/pileup/pileup_trackquality_Mchi-60p0_dMchi-20p0_ctau-100.root


Create pandas dataframes for different physics objects now that uproot won't flatten them anymore
(does make code faster)

In [21]:
leadingJet = OrderedDict({}); MET = OrderedDict({});
muons = OrderedDict({}); vertex = OrderedDict({});
cuts = OrderedDict({}); cutsCrit = OrderedDict({});
checkmuons = OrderedDict({}); gen = OrderedDict({})
checkgmuons = OrderedDict({}); beamhalo = OrderedDict({});
for mchi in mchis:
    leadingJet[mchi] = OrderedDict({}); MET[mchi] = OrderedDict({});
    muons[mchi] = OrderedDict({}); vertex[mchi] = OrderedDict({});
    cuts[mchi] = OrderedDict({}); cutsCrit[mchi] = OrderedDict({});
    checkmuons[mchi] = OrderedDict({}); gen[mchi] = OrderedDict({})
    checkgmuons[mchi] = OrderedDict({}); beamhalo[mchi] = OrderedDict({});
    for ctau in ctaus:
        leadingJet[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoPFJetPt','recoPFJetEta','recoPFJetPhi']).loc[(slice(None),0),slice(None)].reset_index(level=1)
        MET[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoPFMetPt', 'recoPFMetPhi'])#.reset_index(level=1)
        muons[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz']).reset_index(level=1)
        vertex[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoDr','recoVxy','recoVz']).reset_index(level=1)
        cuts[mchi][ctau] = trees[mchi][ctau].pandas.df('cutsVec*')
        cutsCrit[mchi][ctau] = [ cuts[mchi][ctau][f'cutsVec[{cut}]'] == 1 for cut in numCuts ]
        checkmuons[mchi][ctau] = trees[mchi][ctau].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','trackHits','trackPlanes','trackChi2','qualityTrack','isGenMatched','GenDR']).reset_index(level=1)
        checkgmuons[mchi][ctau] = treesglobal[mchi][ctau].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','trackHits','trackPlanes','trackChi2','qualityTrack','isGenMatched','GenDR']).reset_index(level=1)
        beamhalo[mchi][ctau] = treesglobal[mchi][ctau].pandas.df(['fired','beamHaloHcal','beamHaloEcal','beamHaloCSC','beamHaloGlobal','beamHaloGlobalsuper','beamHaloCSCtight','trackHits','trackPlanes','trackChi2','qualityTrack']).reset_index(level=1)

Separately calculate the phi average angle between the two muons -- for use in DeltaPhi(MET, muon pair) -- since it takes a while

In [ ]:
beamhalo['60p0']

In [22]:
for mass in mchis:
    for ctau in ctaus:
        csc = beamhalo[mass][ctau]['beamHaloCSC'].mean()
        hcal = beamhalo[mass][ctau]['beamHaloHcal'].mean()
        ecal = beamhalo[mass][ctau]['beamHaloEcal'].mean()
        glob = beamhalo[mass][ctau]['beamHaloGlobal'].mean()
        globsuper = beamhalo[mass][ctau]['beamHaloGlobalsuper'].mean()
        csctight = beamhalo[mass][ctau]['beamHaloCSCtight'].mean()
        #print(csc,hcal,ecal,glob,globsuper,csctight)
        csc_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloCSC'].mean()
        hcal_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloHcal'].mean()
        ecal_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloEcal'].mean()
        glob_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloGlobal'].mean()
        globsuper_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloGlobalsuper'].mean()
        csctight_trig = beamhalo[mass][ctau][beamhalo[mass][ctau]['fired']==1]['beamHaloCSCtight'].mean()
    
        csc_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloCSC'].mean()
        hcal_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloHcal'].mean()
        ecal_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloEcal'].mean()
        glob_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloGlobal'].mean()
        globsuper_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloGlobalsuper'].mean()
        csctight_qual = beamhalo[mass][ctau][beamhalo[mass][ctau]['qualityTrack']==1]['beamHaloCSCtight'].mean()

        csc_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloCSC'].mean()
        hcal_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloHcal'].mean()
        ecal_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloEcal'].mean()
        glob_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloGlobal'].mean()
        globsuper_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloGlobalsuper'].mean()
        csctight_qual_trig = beamhalo[mass][ctau][(beamhalo[mass][ctau]['qualityTrack']==1) & (beamhalo[mass][ctau]['fired']==1)]['beamHaloCSCtight'].mean()
        #print(csc_trig,hcal_trig,ecal_trig,glob_trig,globsuper_trig,csctight_trig)
        table= [[csc,hcal,ecal,glob,globsuper,csctight],
            [csc_trig,hcal_trig,ecal_trig,glob_trig,globsuper_trig,csctight_trig],
            [csc_qual,hcal_qual,ecal_qual,glob_qual,globsuper_qual,csctight_qual],
            [csc_qual_trig,hcal_qual_trig,ecal_qual_trig,glob_qual_trig,globsuper_qual_trig,csctight_qual_trig]]
        beam= pd.DataFrame(table, columns=['csc','hcal','ecal','glob','globsuper','csctight'])
        print(beam)

        csc      hcal  ecal      glob  globsuper  csctight
0  0.127457  0.150663   0.0  0.003359   0.000087  0.034665
1  0.128267  0.201387   0.0  0.003254   0.000081  0.035946
2  0.113894  0.150520   0.0  0.003221   0.000094  0.031468
3  0.115477  0.202228   0.0  0.003090   0.000075  0.032554
        csc      hcal  ecal      glob  globsuper  csctight
0  0.127602  0.144926   0.0  0.003513   0.000083  0.035375
1  0.124218  0.196726   0.0  0.003334   0.000090  0.035790
2  0.114676  0.144527   0.0  0.003299   0.000065  0.032090
3  0.111923  0.196457   0.0  0.003171   0.000073  0.032592
        csc      hcal  ecal      glob  globsuper  csctight
0  0.129457  0.146686   0.0  0.003557   0.000030  0.035332
1  0.129860  0.199315   0.0  0.003346   0.000011  0.035805
2  0.115114  0.145936   0.0  0.003339   0.000031  0.031604
3  0.115560  0.198648   0.0  0.003141   0.000015  0.032098


In [ ]:
# Helper function to calculate average angles
# This takes a few seconds to run, since we 
# are using the apply method
def calcAvgAngle(group):
    x = np.cos(group['recoPhi'].iloc[0]) + np.cos(group['recoPhi'].iloc[-1])
    y = np.sin(group['recoPhi'].iloc[0]) + np.sin(group['recoPhi'].iloc[-1])
    return math.atan2(y/2, x/2)
avgMuonAngle = OrderedDict({})
for mchi in mchis:
    avgMuonAngle[mchi] = muons[mchi].groupby('entry').apply(calcAvgAngle)

# Plot everything together

To plot individual plots separately, copy relevant block of code and replace "axes[i,j]" with "plt", and take note that some of the functions change name, e.g. axes[i,j].set_xlabel() --> plt.xlabel() and axes[i,j].set_ylim() --> plt.ylim().

The other option is to copy the block and add "fig, ax = plt.subplots(1,1)" at the top, and then do the replacement axes[i,j] --> ax elsewhere.

## Helper functions

In [ ]:
insets = True
log = False
histtype ='step'
iheight="45%"
iwidth="35%"

def plot_inset(data, axis, kwargs={}):
    #if 'bins' not in kwargs: kwargs['bins'] = 10
    if 'histtype' not in kwargs: kwargs['histtype'] = histtype#build
    if insets:
        axins = inset_axes(axis, width=iwidth, height=iheight)
        for (cut,datum) in enumerate(data):
            ret = axins.hist(datum, **kwargs)
            if cut == 1:
                axins.set_ylim(100,1.1*max(ret[0]))
        return axins

def plot_full(data, axis, kwargs={}, labels = []):
    if 'bins' not in kwargs: kwargs['bins'] = 50
    if 'histtype' not in kwargs: kwargs['histtype'] = histtype
    if 'log' not in kwargs: kwargs['log'] = log
    for (cut,datum) in enumerate(data):
        ret = axis.hist(datum, label=(labels[cut] if len(labels) > 0 else ''), **kwargs)
        if cut == 0:
            axis.set_ylim(1, 1.5*max(ret[0]))
            
# Helper function to normalize angle differences to [-Pi, Pi]
# cond: if abs(phidiff) > Pi => phidiff = phidiff - 2*Pi*sign(phidiff)
def reducephi(row):
    if abs(row) > math.pi:
        return row - 2*math.pi*(row/abs(row))
    return row

In [ ]:
mchi = '6p0'

### Make subplot axes
fig, axes = plt.subplots(2,2, figsize=(14,8))
plt.tight_layout(h_pad=4.0, w_pad=16.0)
for row in axes:
    for col in row:
        col.set_ylabel('Events')
        
fig.suptitle(f'Quality and gen matching check for {mchis[mchi]} dsa muons')
fig.subplots_adjust(top=0.92)

genlabels1 = ['all muons','Gen','!Gen']#, 'is gen and quality']   
genlabels2 = ['all muons','quality','!quality']#
genlabels3 = ['all muons','Gen','quality']#
### reco MET pT
axes[0,0].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['isGenMatched']==True]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['isGenMatched']==False]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[0,0], dict({'range':(0,20)}), labels=genlabels1)


axes[0,1].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['qualityTrack']==False]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[0,1], dict({'range':(0,20)}), labels=genlabels2)

axes[1,0].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['isGenMatched']==True]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[1,0], dict({'range':(0,20)}), labels=genlabels3)

genlabels4 = ['all muons','chi2','hits','planes']#
axes[1,1].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <10]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[1,1], dict({'range':(0,20)}), labels=genlabels4)

axes[0,0].legend(loc=(1.05,.3), title='Cuts');
axes[0,1].legend(loc=(1.05,.3), title='Cuts');
axes[1,0].legend(loc=(1.05,.3), title='Cuts');
axes[1,1].legend(loc=(1.05,.3), title='Cuts');

In [ ]:
checkmuons[mchi].groupby('entry').nth(0).corr()

In [ ]:
#checkmuons[mchi]['isGenMatched'].corr(checkmuons[mchi]['qualityTrack'])
#checkmuons[mchi][checkmuons[mchi]['recoPt'] > 4].groupby('entry').nth(0).corr()
checkmuons[mchi][checkmuons[mchi]['isGenMatched']==True].corr()

In [ ]:
mchi = '6p0'

### Make subplot axes
fig, axes = plt.subplots(2,2, figsize=(14,8))
plt.tight_layout(h_pad=4.0, w_pad=16.0)
for row in axes:
    for col in row:
        col.set_ylabel('Events')
        
fig.suptitle(f'Quality and gen matching check for {mchis[mchi]} global muons')
fig.subplots_adjust(top=0.92)

genlabels1 = ['all muons','Gen','!Gen']#, 'is gen and quality']   
genlabels2 = ['all muons','quality','!quality']#
genlabels3 = ['all muons','Gen','quality']#
### reco MET pT
axes[0,0].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkgmuons[mchi]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['isGenMatched']==True]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['isGenMatched']==False]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[0,0], dict({'range':(0,20)}), labels=genlabels1)


axes[0,1].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkgmuons[mchi]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['qualityTrack']==True]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['qualityTrack']==False]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[0,1], dict({'range':(0,20)}), labels=genlabels2)

axes[1,0].set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkgmuons[mchi]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['qualityTrack']==True]['recoPt'])
data.append(checkgmuons[mchi][checkgmuons[mchi]['isGenMatched']==True]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes[1,0], dict({'range':(0,20)}), labels=genlabels3)

axes[0,0].legend(loc=(1.05,.3), title='Cuts');
axes[0,1].legend(loc=(1.05,.3), title='Cuts');
axes[1,0].legend(loc=(1.05,.3), title='Cuts');

In [ ]:
checkgmuons[mchi].groupby('entry').nth(0).corr()

## Plot MET and jet variables

# Scratch tests and comparisons

Comparing results before and after fixing Dxy acceptance (i.e. Dxy is a signed distance and can be < 0)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
chilabels = ['all','<20','<12','<10','<8','<4','<2','<1']#
axes.set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <20]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <10]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <12]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <8]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <4]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <2]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <1]['recoPt'])

#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes, dict({'range':(0,20)}),labels=chilabels)
plot_inset(data, axes, dict({'range':(9,15)}))
axes.legend(loc=(1.05,.3), title='Chi');

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
hitlabels = ['all','>15','>12','>10','>8']#
axes.set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >15]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >12]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >10]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >8]['recoPt'])




#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes, dict({'range':(0,20)}),labels=hitlabels)
plot_inset(data, axes, dict({'range':(9,15)}))

axes.legend(loc=(1.05,.3), title='Hits');

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
planelabels = ['all','>1','>2','>3','>4']#
axes.set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >1]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >2]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >3]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >4]['recoPt'])




#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes, dict({'range':(0,20)}),labels=planelabels)
plot_inset(data, axes, dict({'range':(9,15)}))

axes.legend(loc=(1.05,.3), title='planes');

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
alllabels = ['all','10,8,1','15,8,1','15,6,1','15,4,1','x,x,1','x,x,2']#
axes.set_xlabel('reco Mu pT [GeV]')
data = []
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi]['recoPt'])
data.append(checkmuons[mchi][(checkmuons[mchi]['trackChi2'] <10) & (checkmuons[mchi]['trackHits'] >8) & (checkmuons[mchi]['trackPlanes'] >1)]['recoPt'])
data.append(checkmuons[mchi][(checkmuons[mchi]['trackChi2'] <15) & (checkmuons[mchi]['trackHits'] >8) & (checkmuons[mchi]['trackPlanes'] >1)]['recoPt'])
data.append(checkmuons[mchi][(checkmuons[mchi]['trackChi2'] <15) & (checkmuons[mchi]['trackHits'] >6) & (checkmuons[mchi]['trackPlanes'] >1)]['recoPt'])
data.append(checkmuons[mchi][(checkmuons[mchi]['trackChi2'] <15) & (checkmuons[mchi]['trackHits'] >4) & (checkmuons[mchi]['trackPlanes'] >1)]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >1]['recoPt'])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >2]['recoPt'])






#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes, dict({'range':(0,20)}),labels=alllabels)
plot_inset(data, axes, dict({'range':(9,15)}))

axes.legend(loc=(1.05,.3), title='chi,hits,planes');

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
#planelabels = ['all','>1','>2','>3','>4']#
axes.set_xlabel('GenDR')
data = []
variable = 'GenDR'
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >=1][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >1][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >2][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >3][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes'] >4][variable])

#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >15][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >12][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >10][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits'] >8][variable])

#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <20][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <10][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <12][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <8][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <4][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <2][variable])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackChi2'] <1][variable])

#data.append(checkmuons[mchi][checkmuons[mchi]['trackHits']> 12]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 2]['recoPt'])
#data.append(checkmuons[mchi][checkmuons[mchi][checkmuons[mchi]['qualityTrack']==True]['qualityTrack']==True]['recoPt'])
plot_full(data, axes, dict({'range':(0,6),'log':True}),labels=planelabels)
plot_inset(data, axes, dict({'range':(0,1.),'log':True}))

axes.legend(loc=(1.05,.3), title='planes');

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(14,8))
#planelabels = ['all','>1','>2','>3','>4']#
axes.set_xlabel('eta')
data = []
variable = 'recoEta'
#for cut in numCuts:
#    cutsToApply = reduce(operator.and_, cutsCrit[mchi][0:cut+1])
data.append(checkmuons[mchi][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['isGenMatched']==True][variable])
data.append(checkmuons[mchi][checkmuons[mchi]['trackPlanes']> 1][variable])

plot_full(data, axes, dict({'range':(-4,4),'log':False}),labels=['all','gen Matched','planes>1'])
#plot_inset(data, axes, dict({'range':(0,1),'log':True}))

axes.legend(loc=(1.05,.3), title='planes');